In [1]:
# Important: We are using PIL to read .png files later.
# This was done on purpose to read indexed png files
# in a special way -- only indexes and not map the indexes
# to actual rgb values. This is specific to PASCAL VOC
# dataset data. If you don't want thit type of behaviour
# consider using skimage.io.imread()
from PIL import Image
import numpy as np
import skimage.io as io
import tensorflow as tf
import nibabel as nib

import os, sys

In [2]:
def load_nii(img_path):
    """
    Function to load a 'nii' or 'nii.gz' file, The function returns
    everyting needed to save another 'nii' or 'nii.gz'
    in the same dimensional space, i.e. the affine matrix and the header

    Parameters
    ----------

    img_path: string
    String with the path of the 'nii' or 'nii.gz' image file name.

    Returns
    -------
    Three element, the first is a numpy array of the image values,
    the second is the affine transformation of the image, and the
    last one is the header of the image.
    """
    nimg = nib.load(img_path)
    return nimg.get_data(), nimg.affine, nimg.header


def save_nii(img_path, data, affine, header):
    """
    Function to save a 'nii' or 'nii.gz' file.

    Parameters
    ----------

    img_path: string
    Path to save the image should be ending with '.nii' or '.nii.gz'.

    data: np.array
    Numpy array of the image data.

    affine: list of list or np.array
    The affine transformation to save with the image.

    header: nib.Nifti1Header
    The header that define everything about the data
    (pleasecheck nibabel documentation).
    """
    nimg = nib.Nifti1Image(data, affine=affine, header=header)
    nimg.to_filename(img_path)

In [3]:
# Helper functions for defining tf types
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))



In [4]:

def write_image_annotation_pairs_to_tfrecord(filename_pairs, tfrecords_filename):
    """Writes given image/annotation pairs to the tfrecords file.
    The function reads each image/annotation pair given filenames
    of image and respective annotation and writes it to the tfrecord
    file.
    Parameters
    ----------
    filename_pairs : array of tuples (img_filepath, annotation_filepath)
        Array of tuples of image/annotation filenames
    tfrecords_filename : string
        Tfrecords filename to write the image/annotation pairs
    """
    writer = tf.python_io.TFRecordWriter(tfrecords_filename)

    #for img_path, annotation_path in filename_pairs:
    for img_path,annotation_path in filename_pairs:
        img = np.load(img_path) 
        annotation = np.load(annotation_path)
        #print(annotation.shape)
        #print(annotation)
        

        #img = np.array(Image.open(img_path))
        
        #annotation = np.array(Image.open(annotation_path))
        
        # Unomment this one when working with surgical data
        # annotation = annotation[:, :, 0]

        # The reason to store image sizes was demonstrated
        # in the previous example -- we have to know sizes
        # of images to later read raw serialized string,
        # convert to 1d array and convert to respective
        # shape that image used to have.
        #height = img.shape[0] 
        height = img[0].shape[0] 
        
        #width = img.shape[1]
        width = img[0].shape[1] 
        
        #add depth 
        depth = img[0].shape[2]
        #print(depth)
        #print(img[0].shape) #288,288,140 


        #img_raw = img.tostring()
        img_raw=img[0].tostring()
        annotation_raw = annotation[0].tostring()
        print(annotation[0].shape)
        #print(annotation[1].shape)
        #print(annotation[2].shape)
        
    
        #print(annotation_raw)
        #annotation_raw = annotation.tostring()

        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(height),
            'width': _int64_feature(width),
            'depth': _int64_feature(depth), 
            'image_raw': _bytes_feature(img_raw),
            'mask_raw': _bytes_feature(annotation_raw)}))

        writer.write(example.SerializeToString())

    writer.close()


def read_image_annotation_pairs_from_tfrecord(tfrecords_filename):
    """Return image/annotation pairs from the tfrecords file.
    The function reads the tfrecords file and returns image
    and respective annotation matrices pairs.
    Parameters
    ----------
    tfrecords_filename : string
        filename of .tfrecords file to read from
    
    Returns
    -------
    image_annotation_pairs : array of tuples (img, annotation)
        The image and annotation that were read from the file
    """
    
    image_annotation_pairs = []

    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    for string_record in record_iterator:

        example = tf.train.Example()
        example.ParseFromString(string_record)

        height = int(example.features.feature['height']
                                     .int64_list
                                     .value[0])

        width = int(example.features.feature['width']
                                    .int64_list
                                    .value[0])
        
        depth = int(example.features.feature['depth']
                                    .int64_list
                                    .value[0])

        img_string = (example.features.feature['image_raw']
                                      .bytes_list
                                      .value[0])

        annotation_string = (example.features.feature['mask_raw']
                                    .bytes_list
                                    .value[0])

        img_1d = np.fromstring(img_string, dtype=np.uint8)
        img = img_1d.reshape((height, width, depth, -1))

        annotation_1d = np.fromstring(annotation_string, dtype=np.uint8)

        # Annotations don't have depth (3rd dimension)
        # TODO: check if it works for other datasets
        annotation = annotation_1d.reshape((height, width, depth,-1))

        image_annotation_pairs.append((img, annotation))
    
    return image_annotation_pairs



In [5]:

def read_tfrecord_and_decode_into_image_annotation_pair_tensors(tfrecord_filenames_queue):
    """Return image/annotation tensors that are created by reading tfrecord file.
    The function accepts tfrecord filenames queue as an input which is usually
    can be created using tf.train.string_input_producer() where filename
    is specified with desired number of epochs. This function takes queue
    produced by aforemention tf.train.string_input_producer() and defines
    tensors converted from raw binary representations into
    reshaped image/annotation tensors.
    Parameters
    ----------
    tfrecord_filenames_queue : tfrecord filename queue
        String queue object from tf.train.string_input_producer()
    
    Returns
    -------
    image, annotation : tuple of tf.int32 (image, annotation)
        Tuple of image/annotation tensors
    """
    
    reader = tf.TFRecordReader()

    _, serialized_example = reader.read(tfrecord_filenames_queue)

    features = tf.parse_single_example(
      serialized_example,
      features={
        'height': tf.FixedLenFeature([], tf.int64),
        'width': tf.FixedLenFeature([], tf.int64),
        'depth': tf.FixedLenFeature([], tf.int64), 
        'image_raw': tf.FixedLenFeature([], tf.string),
        'mask_raw': tf.FixedLenFeature([], tf.string)
        })

    
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    annotation = tf.decode_raw(features['mask_raw'], tf.uint8)
    
    height = tf.cast(features['height'], tf.int32)
    width = tf.cast(features['width'], tf.int32) 
    depth = tf.cast(features['depth'],tf.int32)
    
    #image_shape = tf.pack([height, width, depth, 3])
    image_shape = tf.stack([height, width, depth, 3])
    
    # The last dimension was added because
    # the tf.resize_image_with_crop_or_pad() accepts tensors
    # that have depth. We need resize and crop later.
    # TODO: See if it is necessary and probably remove third
    # dimension
    annotation_shape = tf.stack([height, width, depth, 1])
    
    image = tf.reshape(image, image_shape)
    annotation = tf.reshape(annotation, annotation_shape)
    
    return image, annotation

In [6]:
img_path=os.path.abspath('mr_train_1001_image.nii')
label_path=os.path.abspath('mr_train_1001_label.nii')

array_img=load_nii(img_path) 
array_label=load_nii(label_path) 

img_npy_path=os.path.abspath('mr_train_1001_img_npy')
label_npy_path=os.path.abspath('mr_train_1001_label_npy')
    
np.save(img_npy_path, array_img) 
np.save(label_npy_path, array_label) 


img_npy_path=os.path.abspath('mr_train_1001_img_npy.npy')
label_npy_path=os.path.abspath('mr_train_1001_label_npy.npy')

 
f_arr=[[img_npy_path,label_npy_path]]


tfrecords_filename='mr_train_1001_pairs.tfrecords'
write_image_annotation_pairs_to_tfrecord(f_arr, tfrecords_filename)


(512, 512, 160)


In [7]:
pairs=read_image_annotation_pairs_from_tfrecord(tfrecords_filename) 
#print(pairs)

C:\Users\SeojinKim\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
C:\Users\SeojinKim\Anaconda3\lib\site-packages\ipykernel_launcher.py:118: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [8]:
#print(pairs)  
tfrecord_filenames_queue=tf.train.string_input_producer(["mr_train_1001_pairs.tfrecords"])
img,annotation=read_tfrecord_and_decode_into_image_annotation_pair_tensors(tfrecord_filenames_queue)  
print(img)
print(annotation)
#print(output)

Tensor("Reshape:0", shape=(?, ?, ?, 3), dtype=uint8)
Tensor("Reshape_1:0", shape=(?, ?, ?, 1), dtype=uint8)
